In [2]:
from Optimization.Algorithms.Algorithm import Algorithm
from Numerical_Objects.Vector import Vector

import numpy as np 

In [54]:
class DolphinSearch(Algorithm):

    def __init__(self, N, T1, T2, speed, M, A, e):
        self._k = None
        self._f_k = None
        self._TS = np.full((N, N), np.inf)
        self._N = N
        self._T1 = T1
        self._T2 = T2
        self._speed = speed
        self._M = M
        self._A = A
        self._e = e

    @classmethod
    def from_dict(cls, dict_data):
        raise Exception("I don't want to do it and I'll not")

    @classmethod
    def from_json(cls, json_data):
        raise Exception('To be not-json or not to be not-json?')

    def to_dict(self):
        raise Exception('Why?')

    def to_json(self):
        raise Exception('No.')

    @property
    def current_state(self):
        raise Exception("U definitely don't need it!")

    @property
    def iterations(self):
        return [self.get_new_solution,
                # self.information_transfer,
                # self.get_optimal_solution
        ]

    def initialize(self, f, area, seed = None):
        self._k = []
        self._f_k = []
        for i in range(self._N):
            point = {k: np.random.uniform(low,up) for k, (low,up) in area.items()}
            point = Vector(point)
            self._k.append(point)
            self._f_k.append(f(point))

    def  get_new_solution(self, f, area):
        V = []
        solution = []
        fitness = []
        l = []
        fit_l = []
        area_v = {k: (-1, 1) for k, (low, up) in area.items()}

        for i in range(self._M):
            velocity = {k: np.random.uniform(low,up) for k, (low,up) in area_v.items()}
            lgth = Vector(velocity).length
            vec = Vector({k: self._speed * (val / lgth) for k, val in velocity.items()})
            V.append(vec)

        for dol in self._k:
            sol = [dol + v * t for v in V for t in range(1, self._T1 + 1)]
            fit = list(map(f, sol))
            solution.append(sol)
            fitness.append(fit)
        
        for i in range(self._N):
            idx = np.argmin(fitness[i])
            sol = solution[i]
            sol_l = sol[idx]
            l.append(sol_l)
            print(l[i])
            fit_l.append(f(sol_l))
            print(fit_l[i])

            if fit_l[i] < self._f_k[i]:
                self._k[i] = l[i]
                self._f_k[i] = fit_l[i]
        # return self.information_transfer

In [55]:
alg = DolphinSearch(N = 10, T1 = 2, T2 = 3, speed = 5, M = 2, A = 5, e = 2)

In [6]:
def mul(x):
    result = 1
    for val in x.values:
        result = result*val
    return result

In [7]:
area = {'x1': (-10, 10), 'x2': (-10, 10)}

In [62]:
alg.initialize(mul, area)

In [63]:
for k in alg._k:
    print(k)

(x1: 4.2830063226564175) x (x2: -0.4624323828643142)
(x1: -2.6084151129862043) x (x2: -7.771645035955411)
(x1: 8.284582818378933) x (x2: -7.0785437323096945)
(x1: 9.647171213452545) x (x2: -4.249865626137506)
(x1: 1.738462689252227) x (x2: -7.039412471483595)
(x1: 9.593003920288506) x (x2: 6.642690943868136)
(x1: 8.699338050773221) x (x2: 8.283470783336792)
(x1: -6.276282552197903) x (x2: 0.25175020732428344)
(x1: -4.606013105642422) x (x2: 4.627518606764109)
(x1: -9.522133932549355) x (x2: -3.055891560260056)


In [64]:
for f in alg._f_k:
    print(f)

-1.9806008196089309
20.271676364550306
-58.64278178383678
-40.999181329515224
-12.237755935931036
63.72336026579198
72.06071257795001
-1.5800554337416048
-21.31441134935965
29.09860872014347


In [65]:
alg.get_new_solution(mul, area)

(x2: 0.645792525420753) x (x1: -5.655395840299988)
-3.652212361961351
(x2: -3.250295595652916) x (x1: -0.4736085084722692)
1.5393676491511632
(x2: -2.5571942920071997) x (x1: 10.419389422892868)
-26.644403158421834
(x2: -3.6957531719949723) x (x1: 4.677970131974342)
-17.288622953741914
(x2: -2.5180630311811) x (x1: 3.873269293766162)
-9.753136218441501
(x2: 7.750915852153203) x (x1: -0.3453982426678994)
-2.6771527144004805
(x2: 9.391695691621859) x (x1: -1.2390641121831845)
-11.636913084034077
(x2: 1.3599751156093507) x (x1: -16.21468471515431)
-22.051567720061154
(x2: 5.735743515049177) x (x1: -14.544415268598827)
-83.42303555704795
(x2: 5.9868073203449335) x (x1: -5.252520723521485)
-31.445829517841894


In [66]:
for k in alg._k:
    print(k)

(x2: 0.645792525420753) x (x1: -5.655395840299988)
(x2: -3.250295595652916) x (x1: -0.4736085084722692)
(x1: 8.284582818378933) x (x2: -7.0785437323096945)
(x1: 9.647171213452545) x (x2: -4.249865626137506)
(x1: 1.738462689252227) x (x2: -7.039412471483595)
(x2: 7.750915852153203) x (x1: -0.3453982426678994)
(x2: 9.391695691621859) x (x1: -1.2390641121831845)
(x2: 1.3599751156093507) x (x1: -16.21468471515431)
(x2: 5.735743515049177) x (x1: -14.544415268598827)
(x2: 5.9868073203449335) x (x1: -5.252520723521485)
